Picking annotations: making a merged or random category for each multi-AAR option

In [ ]:
#This needs checking as new and old scripts dont give same resutls -- note screen shot from 12/12

In [29]:
import pandas as pd
import os
import numpy as np
import random
import itertools
from collections import Counter
import scanpy as sc
import pickle
pd.set_option("display.max_rows", None, "display.max_columns", None)
pd.options.display.max_colwidth = 200

Take image collection file from Celery app

In [30]:
# labels path
collection_path = "/Users/sanjavickovic/Desktop/colons_comp/R_annotation_scripts"

# empty df to collect all annotations
anns_all_collection = pd.read_csv(os.path.join(collection_path, 'Colon_Dev_annotations_Jan5.txt'), sep = "\t")

In [31]:
start_len = len(anns_all_collection.index)

In [32]:
start_len

97878

Clean up epithelium and lamina propria

In [33]:
anns_all_collection.value = [i.replace("Epithelium - All;Lamina propria", "Epithelium - Lamina Propria") for i in anns_all_collection.value ]
anns_all_collection.value = [i.replace("Epithelium - All;Epithelium - Lamina Propria", "Epithelium - Lamina Propria") for i in anns_all_collection.value]


In [46]:
anns_all_collection.value = [i.replace("Epithelium - All", "Epithelium - Lamina Propria") for i in anns_all_collection.value ]

Do a sanity check to see if clean up worked

In [47]:
news = []
for x in anns_all_collection.value:
    if "Epithelium - All" in x:
        if 'Lamina' in x:
            news.append(x.replace(";Lamina propria", "").replace( "Epithelium - All", "Epithelium - Lamina Propria"))
            
    else:
        news.append(x)
            

In [48]:
len(news)

97878

Do initial renaming

In [49]:
if (len(news) == len(anns_all_collection.value)):
    print("ok")
    anns_all_collection.value = news

ok


In [50]:
len(anns_all_collection.index)

97878

In [64]:
# makes splotch output annotations based on what's been pre-processed for tangram

# # labels path
# labels_path = "/Users/sanjavickovic/Desktop/colons_comp/labels"

# # get path to training directory per image 
# train_path = '/Users/sanjavickovic/Desktop/colons_comp/train'

# #list all dirs in the train_path
# imgs = os.listdir(train_path)
# imgs.remove('.DS_Store') # makes sure Sanja's cloud stuff isn't in here

# # empty df to collect all annotations
# df_anns_all = pd.DataFrame(columns = ['image', 'x_y', 'x', 'y', 'value'])

# # based on 12w data for now 
# anns_all = []
# anns_random = []
# for image_name in imgs:   
    
#     ann_file = image_name + '_labels.csv'
#     ann = pd.read_csv(os.path.join(labels_path, ann_file))
#     splotch_ann = pd.DataFrame(columns = ['image', 'x_y', 'x', 'y', 'value'])
#     splotch_ann['image'] = [i.split("_")[0]+'_'+i.split("_")[1] for i in ann['id']]
#     splotch_ann['x_y'] = [i.split("_")[2]+'_'+i.split("_")[3] for i in ann['id']]
#     splotch_ann['x'] = [i.split("_")[2] for i in ann['id']]
#     splotch_ann['y'] = [i.split("_")[3] for i in ann['id']]
#     splotch_ann['value'] = ann['annotation']
# empty df to collect all annotations
df_anns_all = pd.DataFrame(columns = ['image', 'x_y', 'x', 'y', 'value'])

# based on 12w data for now 
anns_all = []
anns_random = []

counter = 1

for img in np.unique(anns_all_collection['image']):
    
    # subset to only one img
    #print(img)
    ann_df = anns_all_collection[anns_all_collection['image'] == img]
    #print("Initial size: ", len(ann_df.index))
    
    # if ann label "Unknown" remove those spots
    ann_df = ann_df[~ann_df['value'].isin(['Unknown'])]
    
    #print("Filtering unknowns: ", len(ann_df.index))
    unknowns_length = len(ann_df.index)
    
    # rename regions to make sure they survive tf handling
    new_names = [{x:x.lower().replace(" - "," ").replace(";"," and ")} for x in set(ann_df['value'])]
    vals = [v for v in new_names for k,v in v.items()]
    keys = [v for v in new_names for v,k in v.items()]
    ann_df['value'] = ann_df['value'].replace({keys[i]: vals[i] for i in range(len(vals))})
    
    # rename df
    splotch_ann = ann_df.reset_index(drop=True)
    #print("Check before merging: ", len(splotch_ann.index))
    # some clean up 
#     splotch_ann['value'] = splotch_ann['value'].str.replace('epithelium lamina propria and lamina propria', 'epithelium lamina propria')
#     splotch_ann['value'] = splotch_ann['value'].str.replace('epithelium all and lamina propria', 'epithelium lamina propria')
#     splotch_ann['value'] = splotch_ann['value'].str.replace('submucosa vessels', 'submucosa all')
#     #splotch_ann['value'] = splotch_ann['value'].str.replace('lamina propria and muscularis mucosae', 'epithelium lamina propria and muscularis mucosae')
#     splotch_ann['value'] = splotch_ann['value'].str.replace(' and myenteric plexus', "")    
    
    # some clean up 
    splotch_ann['value'] = splotch_ann['value'].str.replace('epithelium lamina propria and lamina propria', 'epithelium lamina propria')
    splotch_ann['value'] = splotch_ann['value'].str.replace('epithelium all and lamina propria', 'epithelium lamina propria')
    #splotch_ann['value'] = splotch_ann['value'].str.replace('epithelium all', 'epithelium lamina propria')
    splotch_ann['value'] = splotch_ann['value'].str.replace('submucosa vessels', 'submucosa all')
    #splotch_ann['value'] = splotch_ann['value'].str.replace('lamina propria and muscularis mucosae', 'epithelium lamina propria and muscularis mucosae')
    splotch_ann['value'] = splotch_ann['value'].str.replace(' and myenteric plexus', "")    
    #splotch_ann['value'] = splotch_ann['value'].str.replace('epithelium base of crypt and epithelium mid crypt and mucosa', 'epithelium and lamina propria and mucosa')
    #splotch_ann['value'] = splotch_ann['value'].str.replace('epithelium and lamina propria and pellet', 'epithelium and lamina propria and mucosa')

    
    # this is ok 
    if len(splotch_ann[splotch_ann['value'] == "lamina propria"]) > 0:
        # print("ovdje")
        ind = splotch_ann[splotch_ann['value'] == "lamina propria"].index
        splotch_ann.at[ind, 'value'] = "epithelium lamina propria"
    
    # this is ok 
    if len(splotch_ann[splotch_ann['value'] == "submucosa all"]) >0:
        ind = splotch_ann[splotch_ann['value'] == "submucosa all"].index
        #print(splotch_ann[splotch_ann['value'] == "submucosa all"])
        splotch_ann.at[ind, 'value'] = "muscularis all and submucosa all"
        
    # this is ok
    if len(splotch_ann[splotch_ann['value'] == "muscularis mucosae"]) >0:
        ind = splotch_ann[splotch_ann['value'] == "muscularis mucosae"].index
        #print(splotch_ann[splotch_ann['value'] == "muscularis mucosae"])
        splotch_ann.at[ind, 'value'] = "muscularis all and submucosa all"
    
    # this is ok
    if len(splotch_ann[splotch_ann['value'] == "muscularis mucosae and submucosa all"]) >0:
        ind = splotch_ann[splotch_ann['value'] == "muscularis mucosae and submucosa all"].index
        splotch_ann.at[ind, 'value'] = "muscularis all and submucosa all" 
        
       
    if len(splotch_ann[splotch_ann['value'] == "epithelium lamina propria and muscularis mucosae and submucosa vessels"]) == 1:
        ind = splotch_ann[splotch_ann['value'] == "epithelium lamina propria and muscularis mucosae and submucosa vessels"].index
        splotch_ann.at[ind, 'value'] = "epithelium lamina propria and muscularis mucosae and submucosa all"
    
    
    if len(splotch_ann[splotch_ann['value'] == "epithelium lamina propria and muscularis propria muscularis externa and muscularis propria muscularis interna and submucosa all"]) == 1:
        ind = splotch_ann[splotch_ann['value'] == "epithelium lamina propria and muscularis propria muscularis externa and muscularis propria muscularis interna and submucosa all"].index
        splotch_ann.at[ind, 'value'] = "epithelium lamina propria and muscularis mucosae and muscularis propria muscularis externa and muscularis propria muscularis interna and submucosa all"

        
    if len(splotch_ann[splotch_ann['value'].str.contains('lamina propria and muscularis mucosae and muscularis propria ')])>0:
        ind = splotch_ann[splotch_ann['value'].str.contains('lamina propria and muscularis mucosae and muscularis propria ')].index
        for j in ind:
            splotch_ann.at[j, 'value'] = "epithelium lamina propria and muscularis mucosae and muscularis propria muscularis externa and muscularis propria muscularis interna and submucosa all"
 
    if len(splotch_ann[splotch_ann['value'].str.contains('epithelium base of crypt and lamina propria and ')])>0:
        ind = splotch_ann[splotch_ann['value'].str.contains('epithelium base of crypt and lamina propria and ')].index
        for j in ind:
            splotch_ann.at[j, 'value'] = "epithelium base of crypt and lamina propria and muscularis mucosae and muscularis propria muscularis externa and muscularis propria muscularis interna and submucosa all"
  
    #this is ok
    if len(splotch_ann[splotch_ann['value'].str.contains('epithelium lamina propria and muscularis propria muscularis interna and ')])>0:
        ind = splotch_ann[splotch_ann['value'].str.contains('epithelium lamina propria and muscularis propria muscularis interna and ')].index
        for j in ind:
            splotch_ann.at[j, 'value'] = "epithelium lamina propria and muscularis mucosae and muscularis propria muscularis externa and muscularis propria muscularis interna and submucosa all"

    #this is ok
    if len(splotch_ann[splotch_ann['value'].str.contains('epithelium base of crypt and epithelium mid crypt and lamina propria')])>0:
        ind = splotch_ann[splotch_ann['value'].str.contains('epithelium base of crypt and epithelium mid crypt and lamina propria')].index
        for j in ind:
            splotch_ann.at[j, 'value'] = "epithelium base of crypt and epithelium mid crypt and lamina propria"

    if len(splotch_ann[splotch_ann['value'].str.contains('epithelium apex of crypt')])>0:
        ind = splotch_ann[splotch_ann['value'].str.contains('epithelium apex of crypt')].index
        for j in ind:
            splotch_ann.at[j, 'value'] = "epithelium apex of crypt and lamina propria"

    #this is ok
    if len(splotch_ann[splotch_ann['value'].str.contains('epithelium lamina propria and muscularis propria muscularis externa and muscularis propria muscularis interna')])>0:
        ind = splotch_ann[splotch_ann['value'].str.contains('epithelium lamina propria and muscularis propria muscularis externa and muscularis propria muscularis interna')].index
        for j in ind:
            splotch_ann.at[j, 'value'] = "epithelium lamina propria and muscularis mucosae and muscularis propria muscularis externa and muscularis propria muscularis interna and submucosa all"
    
    #if len(splotch_ann[splotch_ann['value'] == "epithelium lamina propria and muscularis mucosae and submucosa all"]) == 1:
        #print(splotch_ann[splotch_ann['value'] == "epithelium lamina propria and muscularis mucosae and submucosa all"])             

    #this is ok
    if len(splotch_ann[splotch_ann['value'].str.contains('muscularis propria muscularis interna and submucosa all')])>0:
        ind1 = splotch_ann[splotch_ann['value'].str.contains('muscularis propria muscularis interna and submucosa all')].index
        tmpdf = splotch_ann.iloc[ind1,:]
        tmpdf = tmpdf[~tmpdf['value'].str.contains('epithelium')]
        ind = tmpdf.index
        #print(splotch_ann[splotch_ann['value'].str.contains('muscularis propria muscularis interna and submucosa all')])
        for j in ind:
            splotch_ann.at[j, 'value'] = "muscularis all and submucosa all"
  
    #this is ok
    if len(splotch_ann[splotch_ann['value'].str.contains('epithelium lamina propria and muscularis propria muscularis interna')])>0:
        ind = splotch_ann[splotch_ann['value'].str.contains('epithelium lamina propria and muscularis propria muscularis interna')].index
        for j in ind:
            splotch_ann.at[j, 'value'] = "epithelium lamina propria and muscularis mucosae"  
   
    # this is ok
    if len(splotch_ann[splotch_ann['value'].str.contains('unknown')])>0:
        splotch_ann = splotch_ann[~splotch_ann['value'].str.contains('unknown')]
    
    if len(splotch_ann[splotch_ann['value']==('submucosa vessels')]):
        print("ovdje")
        ind = splotch_ann[splotch_ann['value'] == "submucosa vessels"].index
        splotch_ann.at[ind, 'value'] = "muscularis mucosae and submucosa all"
        
    if len(splotch_ann[splotch_ann['value'].str.contains('epithelium base of crypt and muscularis mucosae and muscularis propria muscularis externa and muscularis propria muscularis interna')])>0:
        ind = splotch_ann[splotch_ann['value'].str.contains('epithelium base of crypt and muscularis mucosae and muscularis propria muscularis externa and muscularis propria muscularis interna')].index
        for j in ind:
            splotch_ann.at[j, 'value'] = "epithelium base of crypt and lamina propria and muscularis mucosae and muscularis propria muscularis externa and muscularis propria muscularis interna and submucosa all"
 
    if len(splotch_ann[splotch_ann['value'].str.contains('epithelium base of crypt and epithelium mid crypt and muscularis')])>0:
        ind = splotch_ann[splotch_ann['value'].str.contains('epithelium base of crypt and epithelium mid crypt and muscularis')].index
        for j in ind:
            splotch_ann.at[j, 'value'] = "epithelium base of crypt and epithelium mid crypt and lamina propria"

    
    # these 3 have been fixed in celery and can be removed later
    #this is ok
    if len(splotch_ann[splotch_ann['value'].str.contains('muscularis mucosae and muscularis propria muscularis interna and submucosa vessels')])>0:
        splotch_ann = splotch_ann[~splotch_ann['value'].str.contains('muscularis mucosae and muscularis propria muscularis interna and submucosa vessels')]
        
    #this is ok
    if len(splotch_ann[splotch_ann['value'].str.contains('epithelium lamina propria and muscularis propria muscularis externa')])>0:
        splotch_ann = splotch_ann[~splotch_ann['value'].str.contains('epithelium lamina propria and muscularis propria muscularis externa')]
 
    #this is ok
    if len(splotch_ann[splotch_ann['value'].str.contains('epithelium lamina propria and submucosa all')])>0:
        splotch_ann = splotch_ann[~splotch_ann['value'].str.contains('epithelium lamina propria and submucosa all')]
  
    
    if len(splotch_ann[splotch_ann['value'].str.contains('epithelium base of crypt and muscularis')])>0:
        ind = splotch_ann[splotch_ann['value'].str.contains('epithelium base of crypt and muscularis')].index
        for j in ind:
            splotch_ann.at[j, 'value'] = "epithelium base of crypt and muscularis mucosae and muscularis propria muscularis interna"
    
    splotch_ann['value'] = splotch_ann['value'].str.replace('epithelium epithelium lamina propria', 'epithelium lamina propria')
    #splotch_ann['value'] = splotch_ann['value'].str.replace('epithelium base of crypt and epithelium mid crypt', 'epithelium base of crypt and epithelium mid crypt and epithelium lamina propria and muscularis mucosae and submucosa all')
    #splotch_ann['value'] = splotch_ann['value'].str.replace('lamina propria and muscularis propria muscularis interna', 'epithelium lamina propria and muscularis propria muscularis interna')              
    
    if len(splotch_ann[splotch_ann['value']==('lamina propria and muscularis propria muscularis interna')]):
        ind = splotch_ann[splotch_ann['value'] == "lamina propria and muscularis propria muscularis interna"].index
        splotch_ann.at[ind, 'value'] = "epithelium lamina propria and muscularis propria muscularis interna"
    
    if len(splotch_ann[splotch_ann['value']==('lamina propria and muscularis mucosae and submucosa all')]):
        ind = splotch_ann[splotch_ann['value'] == "lamina propria and muscularis mucosae and submucosa all"].index
        splotch_ann.at[ind, 'value'] = "epithelium lamina propria and muscularis propria muscularis interna"  
    
    if len(splotch_ann[splotch_ann['value']==('lamina propria and muscularis mucosae')]):
        ind = splotch_ann[splotch_ann['value'] == "lamina propria and muscularis mucosae"].index
        splotch_ann.at[ind, 'value'] = "epithelium lamina propria and muscularis mucosae"

    if len(splotch_ann[splotch_ann['value']==('epithelium mid crypt and lamina propria and muscularis mucosae')]):
        ind = splotch_ann[splotch_ann['value'] == "epithelium mid crypt and lamina propria and muscularis mucosae"].index
        splotch_ann.at[ind, 'value'] = "epithelium mid crypt and lamina propria"

    if len(splotch_ann[splotch_ann['value']==('epithelium base of crypt and submucosa all')]):
        ind = splotch_ann[splotch_ann['value'] == "epithelium base of crypt and submucosa all"].index
        splotch_ann.at[ind, 'value'] = "epithelium base of crypt and muscularis mucosae and muscularis propria muscularis interna"
    
    if len(splotch_ann[splotch_ann['value']==('epithelium base of crypt and epithelium lamina propria and epithelium mid crypt and lamina propria and muscularis mucosae')]):
        ind = splotch_ann[splotch_ann['value'] == "epithelium base of crypt and epithelium lamina propria and epithelium mid crypt and lamina propria and muscularis mucosae"].index
        splotch_ann.at[ind, 'value'] = "epithelium base of crypt and epithelium mid crypt and lamina propria"
      
    if len(splotch_ann[splotch_ann['value']==('muscularis propria muscularis externa and submucosa all')]):
        ind = splotch_ann[splotch_ann['value'] == "muscularis propria muscularis externa and submucosa all"].index
        splotch_ann.at[ind, 'value'] = "muscularis all and submucosa all"

    if len(splotch_ann[splotch_ann['value']==('epithelium lamina propria and muscularis propria muscularis interna')]):
        ind = splotch_ann[splotch_ann['value'] == "epithelium lamina propria and muscularis propria muscularis interna"].index
        splotch_ann.at[ind, 'value'] = "epithelium lamina propria and muscularis mucosae"
    
    if len(splotch_ann[splotch_ann['value']==('epithelium base of crypt and epithelium mid crypt')]):
        ind = splotch_ann[splotch_ann['value'] == "epithelium base of crypt and epithelium mid crypt"].index
        splotch_ann.at[ind, 'value'] = "epithelium base of crypt and epithelium mid crypt and lamina propria"

    if len(splotch_ann[splotch_ann['value']==('epithelium mid crypt')]):
        ind = splotch_ann[splotch_ann['value'] == "epithelium mid crypt"].index
        splotch_ann.at[ind, 'value'] = "epithelium mid crypt and lamina propria"
    
    if len(splotch_ann[splotch_ann['value']==('epithelium mid crypt and muscularis mucosae')]):
        ind = splotch_ann[splotch_ann['value'] == "epithelium mid crypt and muscularis mucosae"].index
        splotch_ann.at[ind, 'value'] = "epithelium mid crypt and lamina propria"
    
    if len(splotch_ann[splotch_ann['value']==('epithelium base of crypt and epithelium lamina propria and muscularis mucosae and submucosa all')]):
        ind = splotch_ann[splotch_ann['value'] == "epithelium base of crypt and epithelium lamina propria and muscularis mucosae and submucosa all"].index
        splotch_ann.at[ind, 'value'] = "epithelium base of crypt and lamina propria and muscularis mucosae and muscularis propria muscularis externa and muscularis propria muscularis interna and submucosa all"
    
    if len(splotch_ann[splotch_ann['value']==('epithelium base of crypt and epithelium lamina propria')]):
        ind = splotch_ann[splotch_ann['value'] == "epithelium base of crypt and epithelium lamina propria"].index
        splotch_ann.at[ind, 'value'] = "epithelium base of crypt and muscularis mucosae and muscularis propria muscularis interna"
    
    if len(splotch_ann[splotch_ann['value']==('epithelium base of crypt')]):
        ind = splotch_ann[splotch_ann['value'] == "epithelium base of crypt"].index
        splotch_ann.at[ind, 'value'] = "epithelium base of crypt and muscularis mucosae and muscularis propria muscularis interna"

    if len(splotch_ann[splotch_ann['value']==("muscularis mucosae and muscularis propria muscularis externa and muscularis propria muscularis interna and peyer's patch")]):
        ind = splotch_ann[splotch_ann['value'] == "muscularis mucosae and muscularis propria muscularis externa and muscularis propria muscularis interna and peyer's patch"].index
        splotch_ann.at[ind, 'value'] = "muscularis mucosae and muscularis propria muscularis interna and peyer's patch"

    if len(splotch_ann[splotch_ann['value']==("muscularis propria muscularis interna and peyer's patch")]):
        ind = splotch_ann[splotch_ann['value'] == "muscularis propria muscularis interna and peyer's patch"].index
        splotch_ann.at[ind, 'value'] = "muscularis mucosae and muscularis propria muscularis interna and peyer's patch"
        
    if len(splotch_ann[splotch_ann['value']==("muscularis propria muscularis externa and muscularis propria muscularis interna and peyer's patch")]):
        ind = splotch_ann[splotch_ann['value'] == "muscularis propria muscularis externa and muscularis propria muscularis interna and peyer's patch"].index
        splotch_ann.at[ind, 'value'] = "muscularis mucosae and muscularis propria muscularis interna and peyer's patch"
    
    if len(splotch_ann[splotch_ann['value']==("epithelium lamina propria and muscularis mucosae and peyer's patch")]):
        ind = splotch_ann[splotch_ann['value'] == "epithelium lamina propria and muscularis mucosae and peyer's patch"].index
        splotch_ann.at[ind, 'value'] = "epithelium and lamina propria and peyer's patch"
       
        
    if len(splotch_ann[splotch_ann['value'].str.contains("and peyer's patch and submucosa all")])>0:
        ind = splotch_ann[splotch_ann['value'].str.contains("and peyer's patch and submucosa all")].index
        for j in ind:
            splotch_ann.at[j, 'value'] = "muscularis mucosae and muscularis propria muscularis interna and peyer's patch"

   
            
    ############ final cleanup ##################    
    splotch_ann['value'] = splotch_ann['value'].str.replace('epithelium lamina propria', 'epithelium and lamina propria')
    
    if len(splotch_ann[splotch_ann['value']==("epithelium and lamina propria and epithelium mid crypt and lamina propria")]):
        ind = splotch_ann[splotch_ann['value'] == "epithelium and lamina propria and epithelium mid crypt and lamina propria"].index
        splotch_ann.at[ind, 'value'] = "epithelium base of crypt and epithelium mid crypt and lamina propria"
          
    if len(splotch_ann[splotch_ann['value']==("epithelium and lamina propria and epithelium base of crypt")]):
        ind = splotch_ann[splotch_ann['value'] == "epithelium and lamina propria and epithelium base of crypt"].index
        splotch_ann.at[ind, 'value'] = "epithelium base of crypt and lamina propria"
           
    if len(splotch_ann[splotch_ann['value']==("epithelium base of crypt and epithelium and lamina propria and epithelium mid crypt and muscularis mucosae")]):
        ind = splotch_ann[splotch_ann['value'] == "epithelium base of crypt and epithelium and lamina propria and epithelium mid crypt and muscularis mucosae"].index
        splotch_ann.at[ind, 'value'] = "epithelium base of crypt and epithelium mid crypt and lamina propria"
    
    if len(splotch_ann[splotch_ann['value']==("epithelium base of crypt and epithelium and lamina propria and epithelium mid crypt")]):
        ind = splotch_ann[splotch_ann['value'] == "epithelium base of crypt and epithelium and lamina propria and epithelium mid crypt"].index
        splotch_ann.at[ind, 'value'] = "epithelium base of crypt and epithelium mid crypt and lamina propria"   

        
    if len(splotch_ann[splotch_ann['value']==("epithelium and lamina propria and epithelium base of crypt and epithelium mid crypt")]):
        ind = splotch_ann[splotch_ann['value'] == "epithelium and lamina propria and epithelium base of crypt and epithelium mid crypt"].index
        splotch_ann.at[ind, 'value'] = "epithelium base of crypt and epithelium mid crypt and lamina propria"

    if len(splotch_ann[splotch_ann['value']==("epithelium and lamina propria and epithelium mid crypt and lamina propria")]):
        ind = splotch_ann[splotch_ann['value'] == "epithelium and lamina propria and epithelium mid crypt and lamina propria"].index
        splotch_ann.at[ind, 'value'] = "epithelium mid crypt and lamina propria"

    if len(splotch_ann[splotch_ann['value']==("epithelium and lamina propria and epithelium mid crypt")]):
        ind = splotch_ann[splotch_ann['value'] == "epithelium and lamina propria and epithelium mid crypt"].index
        splotch_ann.at[ind, 'value'] = "epithelium mid crypt and lamina propria"

    if len(splotch_ann[splotch_ann['value']==("epithelium base of crypt and epithelium and lamina propria and muscularis mucosae")]):
        ind = splotch_ann[splotch_ann['value'] == "epithelium base of crypt and epithelium and lamina propria and muscularis mucosae"].index
        splotch_ann.at[ind, 'value'] = "epithelium base of crypt and lamina propria and muscularis mucosae and muscularis propria muscularis externa and muscularis propria muscularis interna and submucosa all"
           
    if len(splotch_ann[splotch_ann['value']==("epithelium base of crypt and lamina propria")]):
        ind = splotch_ann[splotch_ann['value'] == "epithelium base of crypt and lamina propria"].index
        splotch_ann.at[ind, 'value'] = "epithelium base of crypt and lamina propria and muscularis mucosae and muscularis propria muscularis externa and muscularis propria muscularis interna and submucosa all"
    
    if len(splotch_ann[splotch_ann['value']==("epithelium and lamina propria and epithelium base of crypt")]):
        ind = splotch_ann[splotch_ann['value'] == "epithelium and lamina propria and epithelium base of crypt"].index
        splotch_ann.at[ind, 'value'] = "epithelium base of crypt and lamina propria and muscularis mucosae and muscularis propria muscularis externa and muscularis propria muscularis interna and submucosa all"

    if len(splotch_ann[splotch_ann['value']==("epithelium base of crypt and epithelium and lamina propria and epithelium mid crypt and muscularis mucosae and muscularis propria muscularis interna")]):
        ind = splotch_ann[splotch_ann['value'] == "epithelium base of crypt and epithelium and lamina propria and epithelium mid crypt and muscularis mucosae and muscularis propria muscularis interna"].index
        splotch_ann.at[ind, 'value'] = "epithelium base of crypt and epithelium mid crypt and lamina propria"

    # comment this to have an extra layer
    splotch_ann['value'] = splotch_ann['value'].str.replace('muscularis propria muscularis externa and muscularis propria muscularis interna and muscularis mucosae','muscularis propria muscularis externa and muscularis propria muscularis interna')
    splotch_ann['x'] = [i.split("_")[0] for i in splotch_ann['x_y']]
    splotch_ann['y'] = [i.split("_")[1] for i in splotch_ann['x_y']]
    
    # write file used as input into Britta's splotch pre-processing script
    splotch_ann_patch = "/Users/svickovi/Desktop/colons_comp/splotch_ann/merged"
    splotch_ann_file = img + '_annotations.txt'
    #splotch_ann_file = image_name + '_annotations.txt'
    #splotch_ann.to_csv(os.path.join(splotch_ann_patch, splotch_ann_file), index = False, sep = "\t") # uncomment to create new annotation files
                                 
    # collect data into one df ann
    df_anns_all = pd.concat([df_anns_all, splotch_ann])
   
    # collect a list for checking
    anns_all.append(splotch_ann['value'].tolist())    
    
    # write file used as input into Britta's splotch pre-processing script
    # random picking
    
    l = list()
    for n in splotch_ann['value'].tolist():
        if (len(n.split(' and ')) > 0):
            l.append(random.choice(n.split(' and ')))
            
    splotch_ann['value'] = l
    splotch_ann_patch = "/Users/svickovi/Desktop/colons_comp/splotch_ann/random"
    #splotch_ann.to_csv(os.path.join(splotch_ann_patch, splotch_ann_file), index = False, sep = "\t") # uncomment if you want random annotaitons files
    
    #print("Last check after merging: ", len(splotch_ann.index))
    
    # check number of spots lost in merging
    if abs(unknowns_length-len(splotch_ann.index)) > 3:
        print(img)
        print("Too many spots lost in merging...", abs(unknowns_length-len(splotch_ann.index)))
    
    anns_random.append(splotch_ann['value'].tolist())
    
    #if counter > 10:
    #    break
    
    counter += 1
    
#df_anns_all.to_csv(os.path.join("/Users/sanjavickovic/Desktop/colons_comp/splotch_ann/", "splotch_annotations_merged.tsv"), index = False, sep = "\t")                          

10007CN71_C2
Too many spots lost in merging... 30
10007CN71_D1
Too many spots lost in merging... 10
10007CN71_D2
Too many spots lost in merging... 5
10007CN72_C1
Too many spots lost in merging... 20
10007CN72_C2
Too many spots lost in merging... 10
L9CN81_E2
Too many spots lost in merging... 8


In [66]:
# list total annotations in the whole dataset as check 
a = list(itertools.chain.from_iterable(anns_all))
df = pd.DataFrame(Counter(a).values(),Counter(a).keys())
df.columns = ['value']
df.sort_values(by=['value'])

value
muscularis mucosae and peyer's patch                                                                                                                                         48
epithelium and lamina propria and peyer's patch                                                                                                                              55
muscularis mucosae and muscularis propria muscularis interna and peyer's patch                                                                                               87
epithelium base of crypt and muscularis mucosae and muscularis propria muscularis interna                                                                                    90
peyer's patch                                                                                                                                                               312
epithelium mid crypt and lamina propria                                                                                                                                     374
epithelium base of crypt and lamina propria and muscularis mucosae and muscularis propria muscularis externa and muscularis propria muscularis interna and submucosa all    485
muscularis mucosae and muscularis propria muscularis externa and muscularis propria muscularis interna                                                                      556
muscularis mucosae and muscularis propria muscularis interna                                                                                                                583
epithelium base of crypt and epithelium mid crypt and lamina propria                                                                                                        991
epithelium and lamina propria and muscularis mucosae and submucosa all                                                                                                     1589
epithelium apex of crypt and lamina propria                                                                                                                                1843
pellet                                                                                                                                                                     3398
muscularis all and submucosa all                                                                                                                                           3484
muscularis propria muscularis externa                                                                                                                                      3653
muscularis propria muscularis interna                                                                                                                                      3997
epithelium and lamina propria and muscularis mucosae                                                                                                                       7422
epithelium and lamina propria and muscularis mucosae and muscularis propria muscularis externa and muscularis propria muscularis interna and submucosa all                 9102
muscularis propria muscularis externa and muscularis propria muscularis interna                                                                                           10268
epithelium and lamina propria                                                                                                                                             36218

Merge cleaned df with cleaned up/manual sanja annotation txt file for final anns

In [67]:
sanns = pd.read_csv('/Users/sanjavickovic/Desktop/corrected_annotations.txt', sep = '\t')
sanns.drop_duplicates(inplace = True)
sanns = sanns.reset_index(drop=True)
df_anns_all['image_x_y'] = [i+"_"+j for i,j in zip(df_anns_all['image'], df_anns_all['x_y'])]

#merge
pd.merge(sanns, df_anns_all, left_on = 'image_x_y', right_on = 'image_x_y')
pd_changed = pd.merge(sanns, df_anns_all, left_on = 'image_x_y', right_on = 'image_x_y', how = 'left')
pd_changed = pd_changed.drop(pd_changed[pd_changed.image_x_y.str.contains('10015CN91_C1')].index, axis = 0).reset_index(drop=True)
pd_unchanged = pd.merge(sanns, df_anns_all, left_on = 'image_x_y', right_on = 'image_x_y', how = 'right')
pd_unchanged = pd_unchanged[pd_unchanged.isnull().any(axis=1)].reset_index(drop=True)
pd_unchanged['value_x'] = pd_unchanged['value_y']

#make new file
pd_anns_final = pd.concat([pd_unchanged,pd_changed]).drop(['image_x_y', 'value_y'], axis = 1).reset_index(drop=True).rename(columns={'value_x':'value'})
pd_anns_final = pd_anns_final[pd_anns_final['value'] != 'Unknown'].reset_index(drop=True)


Rename patches to user-friendly version

In [68]:
rename_dict = {"muscularis mucosae and muscularis propria muscularis interna and peyer's patch":"mucosae and peyer's patch",
               "epithelium and peyer's patch":"peyer's patch",
                  'epithelium and lamina propria' : 'epithelium',
    'epithelium and lamina propria and mucus' : 'epithelium and mucus',
    'epithelium and lamina propria and mucus and pellet': 'epithelium and mucus and pellet',
    'epithelium and lamina propria and muscularis mucosae' : 'epithelium and mucosae',
    'epithelium and lamina propria and muscularis mucosae and muscularis propria muscularis externa and muscularis propria muscularis interna and submucosa all':'epithelium and muscle and submucosa',
    'epithelium and lamina propria and muscularis mucosae and submucosa all':'epithelium and mucosae and submucosa',
    "epithelium and lamina propria and peyer's patch" : "peyer's patch",
    'epithelium apex of crypt and lamina propria' : 'crypt apex',
    'epithelium base of crypt and epithelium mid crypt and lamina propria' : 'crypt base and mid',
    'epithelium base of crypt and lamina propria' : 'crypt base',
    'epithelium base of crypt and lamina propria and muscularis mucosae and muscularis propria muscularis externa and muscularis propria muscularis interna and submucosa all' : 'crypt base',
    'epithelium base of crypt and muscularis mucosae and muscularis propria muscularis interna': 'crypt base',
    'epithelium mid crypt and lamina propria' : 'crypt mid',
    'mucus and pellet' : 'mucus and pellet',
    'muscularis all and submucosa all' : 'muscle and submucosa',
    'muscularis mucosae and muscularis propria muscularis externa and muscularis propria muscularis interna' : 'muscle and submucosa',
    'muscularis mucosae and muscularis propria muscularis interna' : 'mucosae and interna',
    "muscularis mucosae and muscularis propria muscularis interna and peyer's patch" : "peyer's patch",
    "muscularis mucosae and peyer's patch" : "peyer's patch",
    'muscularis propria muscularis externa' : 'externa',
    'muscularis propria muscularis externa and muscularis propria muscularis interna':'externa and interna',
    'muscularis propria muscularis interna':'interna',
    'muscularis propria muscularis externa and muscularis propria muscularis interna and muscularis mucosae and submucosa all' : 'muscle and submucosa',
    'pellet':'pellet',
    "peyer's patch":"peyer's patch",
    "Rest": "rest"}



In [69]:
import numpy as np
np.unique(list(rename_dict.values()))

array(['crypt apex', 'crypt base', 'crypt base and mid', 'crypt mid',
       'epithelium', 'epithelium and mucosae',
       'epithelium and mucosae and submucosa', 'epithelium and mucus',
       'epithelium and mucus and pellet',
       'epithelium and muscle and submucosa', 'externa',
       'externa and interna', 'interna', 'mucosae and interna',
       'mucus and pellet', 'muscle and submucosa', 'pellet',
       "peyer's patch", 'rest'], dtype='<U36')

In [70]:
#rename dict check
pd_anns_final['value'] = pd_anns_final['value'].map(rename_dict)
#pd_anns_final.to_csv(os.path.join("/Users/sanjavickovic/Desktop/colons_comp/splotch_ann/", "splotch_annotations_merged.tsv"), index = False, sep = "\t")       

In [71]:
pd_anns_final[['value']].value_counts()

value                               
epithelium                              35249
externa and interna                     10714
epithelium and muscle and submucosa      9119
epithelium and mucosae                   6874
muscle and submucosa                     3864
interna                                  3562
pellet                                   3395
externa                                  2964
crypt apex                               2226
epithelium and mucosae and submucosa     1493
crypt base and mid                       1420
mucosae and interna                      1341
crypt mid                                 914
crypt base                                655
peyer's patch                             498
dtype: int64

Output all final ann splotch files

In [72]:
imgs_dfs = np.unique(list(pd_anns_final['image']))
for image_name in imgs_dfs:
    
    # write file used as input into Britta's splotch pre-processing script
    splotch_ann_patch = "/Users/sanjavickovic/Desktop/colons_comp/splotch_ann/merged/jan5" #output dir
    splotch_ann_file = image_name + '_annotations.txt'
    anns_small = pd_anns_final[pd_anns_final['image'] == image_name].reset_index(drop=True)
    anns_small.to_csv(os.path.join(splotch_ann_patch, splotch_ann_file), index = False, sep = "\t") # uncomment to output files


In [54]:
pd_anns_final[['value']].value_counts()

value           
pellet              464
peyer's patch       444
mucus and pellet     29
dtype: int64

In [11]:
img

'10005CN13_C1'